# scipy.optimize examples in cminpack_numba

This notebook contains the examples from the relevant scipy.optimize docstrings and
their equivalent implementations in cminpack_numba


In [1]:
import numpy as np
from numba import cfunc, farray
from numpy.typing import NDArray
from scipy import optimize

import cminpack_numba

## Lmdif & Lmder

### scipy.optimize.least_squares

In [2]:
def func_least_squares(x: NDArray[np.float64]) -> NDArray[np.float64]:
    return np.array([10 * (x[1] - x[0] ** 2), (1 - x[0])])


def jac_least_squares(x: NDArray[np.float64]) -> NDArray[np.float64]:
    return np.array([[-20 * x[0], 10], [-1, 0]])


print(optimize.least_squares(func_least_squares, [2, 2], method="lm").x)
print(
    optimize.least_squares(
        func_least_squares, [2, 2], jac=jac_least_squares, method="lm"
    ).x
)

[1. 1.]
[1. 1.]


In [3]:
@cfunc(cminpack_numba.lmdif_sig)
def func_least_squares_cminpack_numba(udata, m, n, x, fvec, iflag):
    fvec[0] = 10.0 * (x[1] - x[0] ** 2)
    fvec[1] = 1.0 - x[0]
    return 0


@cfunc(cminpack_numba.lmder_sig)
def jac_least_squares_cminpack_numba(udata, m, n, x, fvec, fjac, ldfjac, iflag):
    if iflag == 1:
        fvec[0] = 10.0 * (x[1] - x[0] ** 2)
        fvec[1] = 1.0 - x[0]
    if iflag == 2:
        fjac = farray(fjac, (m, n))
        fjac[0, 0] = -20.0 * x[0]
        fjac[0, 1] = 10.0
        fjac[1, 0] = -1.0
        fjac[1, 1] = 0.0
    return 0


print(
    cminpack_numba.lmdif1(func_least_squares_cminpack_numba.address, 2, 2 * np.ones(2))
)
print(
    cminpack_numba.lmder1(jac_least_squares_cminpack_numba.address, 2, 2 * np.ones(2))
)

(array([1., 1.]), array([0., 0.]), 2)
(array([1., 1.]), array([0., 0.]), array([[20.02498439,  0.04993762],
       [-9.98752339,  0.49937617]]), array([1, 2], dtype=int32), 2)


In [4]:
%timeit optimize.least_squares(func_least_squares, [2, 2], method="lm")
%timeit optimize.least_squares(func_least_squares, [2, 2], jac=jac_least_squares, method="lm")
%timeit cminpack_numba.lmdif1(func_least_squares_cminpack_numba.address, 2, 2*np.ones(2))
%timeit cminpack_numba.lmder1(jac_least_squares_cminpack_numba.address, 2, 2*np.ones(2))

97.7 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
47.9 µs ± 138 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
3.31 µs ± 9.47 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
3.74 µs ± 8.72 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### scipy.optimize.leastsq

In [5]:
def func_leastsq(x):
    return 2 * (x - 3) ** 2 + 1


def jac_leastsq(x):
    return 4 * (x - 3)


print(optimize.leastsq(func_leastsq, 0))
print(optimize.leastsq(func_leastsq, 0, Dfun=jac_leastsq))

(array([2.99999999]), 1)
(array([3.00000001]), 1)


In [6]:
@cfunc(cminpack_numba.lmdif_sig)
def func_leastsq_cminpack_numba(udata, m, n, x, fvec, iflag):
    fvec[0] = 2 * (x[0] - 3) ** 2 + 1
    return 0


@cfunc(cminpack_numba.lmder_sig)
def jac_leastsq_cminpack_numba(udata, m, n, x, fvec, fjac, ldfjac, iflag):
    if iflag == 1:
        fvec[0] = 2 * (x[0] - 3) ** 2 + 1
    if iflag == 2:
        fjac = farray(fjac, (m, n))
        fjac[0, 0] = 4 * (x[0] - 3)
    return 0


print(cminpack_numba.lmdif1(func_leastsq_cminpack_numba.address, 1, np.zeros(1)))
print(cminpack_numba.lmder1(jac_leastsq_cminpack_numba.address, 1, np.zeros(1)))

(array([2.99999999]), array([1.]), 1)
(array([3.00000001]), array([1.]), array([[-2.92966735e-08]]), array([1], dtype=int32), 1)


In [7]:
%timeit optimize.leastsq(func_leastsq, 0)
%timeit optimize.leastsq(func_leastsq, 0, Dfun=jac_leastsq)
%timeit cminpack_numba.lmdif1(func_leastsq_cminpack_numba.address, 1, np.zeros(1))
%timeit cminpack_numba.lmder1(jac_leastsq_cminpack_numba.address, 1, np.zeros(1))

68.7 µs ± 3.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
62.9 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
3.24 µs ± 15.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
3.85 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## hybrd & hybrj

### scipy.optimize.fsolve

In [8]:
def func_fsolve(x):
    return [x[0] * np.cos(x[1]) - 4, x[1] * x[0] - x[1] - 5]


def jac_fsolve(x):
    return [[np.cos(x[1]), -x[0] * np.sin(x[1])], [x[1], x[0] - 1]]


print(optimize.fsolve(func_fsolve, [1, 1]))
print(optimize.fsolve(func_fsolve, [1, 1], fprime=jac_fsolve))

[6.50409711 0.90841421]
[6.50409711 0.90841421]


In [9]:
@cfunc(cminpack_numba.hybrd_sig)
def func_fsolve_cminpack_numba(udata, n, x, fvec, iflag):
    fvec[0] = x[0] * np.cos(x[1]) - 4
    fvec[1] = x[1] * x[0] - x[1] - 5
    return 0


@cfunc(cminpack_numba.hybrj_sig)
def jac_fsolve_cminpack_numba(udata, n, x, fvec, fjac, ldfjac, iflag):
    if iflag == 1:
        fvec[0] = x[0] * np.cos(x[1]) - 4
        fvec[1] = x[1] * x[0] - x[1] - 5
    if iflag == 2:
        fjac = farray(fjac, (n, n))
        fjac[0, 0] = np.cos(x[1])
        fjac[0, 1] = -x[0] * np.sin(x[1])
        fjac[1, 0] = x[1]
        fjac[1, 1] = x[0] - 1
    return 0


print(cminpack_numba.hybrd1(func_fsolve_cminpack_numba.address, np.ones(2))[0])
print(cminpack_numba.hybrj1(jac_fsolve_cminpack_numba.address, np.ones(2))[0])

[6.50409711 0.90841421]
[6.50409711 0.90841421]


In [10]:
%timeit optimize.fsolve(func_fsolve, [1, 1])
%timeit optimize.fsolve(func_fsolve, [1, 1], fprime=jac_fsolve)
%timeit cminpack_numba.hybrd1(func_fsolve_cminpack_numba.address, np.ones(2))
%timeit cminpack_numba.hybrj1(jac_fsolve_cminpack_numba.address, np.ones(2))

30 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
32.6 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
3.71 µs ± 37.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
3.95 µs ± 68.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### scipy.optimize.root

In [11]:
def func_root(x):
    return [x[0] + 0.5 * (x[0] - x[1]) ** 3 - 1.0, 0.5 * (x[1] - x[0]) ** 3 + x[1]]


def jac_root(x):
    return np.array(
        [
            [1 + 1.5 * (x[0] - x[1]) ** 2, -1.5 * (x[0] - x[1]) ** 2],
            [-1.5 * (x[1] - x[0]) ** 2, 1 + 1.5 * (x[1] - x[0]) ** 2],
        ]
    )


print(optimize.root(func_root, [0, 0], method="hybr").x)
print(optimize.root(func_root, [0, 0], jac=jac_root, method="hybr").x)

[0.8411639 0.1588361]
[0.8411639 0.1588361]


In [12]:
@cfunc(cminpack_numba.hybrd_sig)
def func_root_cminpack_numba(udata, n, x, fvec, iflag):
    fvec[0] = x[0] + 0.5 * (x[0] - x[1]) ** 3 - 1.0
    fvec[1] = 0.5 * (x[1] - x[0]) ** 3 + x[1]
    return 0


@cfunc(cminpack_numba.hybrj_sig)
def jac_root_cminpack_numba(udata, n, x, fvec, fjac, ldfjac, iflag):
    if iflag == 1:  # Calculate the function
        fvec[0] = x[0] + 0.5 * (x[0] - x[1]) ** 3 - 1.0
        fvec[1] = 0.5 * (x[1] - x[0]) ** 3 + x[1]
    elif iflag == 2:  # Calculate the Jacobian
        fjac = farray(fjac, (n, n), np.float64)
        fjac[0, 0] = 1 + 1.5 * (x[0] - x[1]) ** 2
        fjac[0, 1] = -1.5 * (x[0] - x[1]) ** 2
        fjac[1, 0] = -1.5 * (x[1] - x[0]) ** 2
        fjac[1, 1] = 1 + 1.5 * (x[1] - x[0]) ** 2
    return 0


print(cminpack_numba.hybrd1(func_root_cminpack_numba.address, np.zeros(2))[0])
print(cminpack_numba.hybrj1(jac_root_cminpack_numba.address, np.zeros(2))[0])

[0.8411639 0.1588361]
[0.8411639 0.1588361]


In [13]:
%timeit optimize.root(func_root, [0, 0], method="hybr")
%timeit optimize.root(func_root, [0, 0], jac=jac_root, method="hybr")
%timeit cminpack_numba.hybrd1(func_root_cminpack_numba.address, np.zeros(2))
%timeit cminpack_numba.hybrj1(jac_root_cminpack_numba.address,  np.zeros(2))

21.7 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
27.9 µs ± 1.72 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
2.47 µs ± 22.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.8 µs ± 15.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
